In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from banditrec.experiments import load_datasets, Experiment, ExperimentVisualizer
from banditrec.agents import UCBAgent, LinUCBAgent, NUCBAgent, NLinUCBAgent, TSAgent, SCLUBAgent
from banditrec.estimators import C3Estimator

## Load datasets and helper functions

In [ ]:
datasets = load_datasets("./data")

exp = Experiment("experiments/raw")
exp.set_run_count(1)

def r1(agent, name=None):
    ds = [datasets['lastfm'], datasets['delicious']]
    vs = ["SW", "RP"]
    exp.run_multiple(ds, vs, agent, name)

def r2(agent, name=None):
    ds = [datasets['lastfmx'], datasets['deliciousx']]
    vs = ["SW", "RP"]
    exp.run_multiple(ds, vs, agent, name)

## Experiments

In [ ]:
# baselines
r1("Random")
r1("Hindsight")
r1(UCBAgent(0.1), "UCB")
r1(NUCBAgent(0.1, 100, 100), "NUCB")
r2("Random")
r2("Hindsight")
r2(LinUCBAgent(0.1, False), "ONE-LinUCB")
r2(LinUCBAgent(0.1, True), "MUL-LinUCB")
r2(NLinUCBAgent(0.1, 100), "NLinUCB")

# experiment2: test effect of global information
r1(NUCBAgent(0.1, 100, 100).toggle_global_info(False), "NUCBl")
r2(NLinUCBAgent(0.1, 100).toggle_global_info(False), "NLinUCBl")

# experiment3: test effect of normalization
r1(NUCBAgent(0.1, 100, 100).toggle_normalization(False), "NUCBn")
r2(NLinUCBAgent(0.1, 100).toggle_normalization(False), "NLinUCBn")
r1(NUCBAgent(0.1, 100, 0).toggle_normalization(False), "NUCBn100")
r1(NUCBAgent(0.1, 200, 0).toggle_normalization(False), "NUCBn200")
r1(NUCBAgent(0.1, 500, 0).toggle_normalization(False), "NUCBn500")
r1(NUCBAgent(0.1, 1000, 0).toggle_normalization(False), "NUCBn1000")
r1(NUCBAgent(0.1, 10000, 0).toggle_normalization(False), "NUCBn10000")

# experiment4: test effect of user versus item weight
r1(NUCBAgent(0.1, 100, 0), "NUCBu100")
r1(NUCBAgent(0.1, 0, 100), "NUCBi100")
r1(NUCBAgent(0.1, 10, 0), "NUCBu10")
r1(NUCBAgent(0.1, 50, 0), "NUCBu50")
r1(NUCBAgent(0.1, 200, 0), "NUCBu200")
r1(NUCBAgent(0.1, 500, 0), "NUCBu500")
r1(NUCBAgent(0.1, 1000, 0), "NUCBu1000")
r1(NUCBAgent(0.1, 10000, 0), "NUCBu10000")
r1(NUCBAgent(0.1, 0, 10), "NUCBi10")
r1(NUCBAgent(0.1, 0, 50), "NUCBi50")
r1(NUCBAgent(0.1, 0, 200), "NUCBi200")
r1(NUCBAgent(0.1, 0, 500), "NUCBi500")
r1(NUCBAgent(0.1, 0, 1000), "NUCBi1000")

r2(NLinUCBAgent(0.1, 50), "NLinUCBu50")
r2(NLinUCBAgent(0.1, 200), "NLinUCBu200")
r2(NLinUCBAgent(0.1, 500), "NLinUCBu500")
r2(NLinUCBAgent(0.1, 1000), "NLinUCBu1000")

# experiment5: test exact algorithm
NUCBe = NUCBAgent(0.1, 100, 0).toggle_exact_version(True)
NLinUCBe = NLinUCBAgent(0.1, 100).toggle_exact_version(True)
r1(NUCBe, "NUCBe")
r2(NLinUCBe, "NLinUCBe")

# experiment6: test user limiting
r1(NUCBe.set_user_limit(100, 'random'), "NUCBe100r")
r1(NUCBe.set_user_limit(500, 'random'), "NUCBe500r")
r1(NUCBe.set_user_limit(100, 'click_window'), "NUCBe100c")
r1(NUCBe.set_user_limit(500, 'click_window'), "NUCBe500c")
r1(NUCBe.set_user_limit(100, 'imp_window'), "NUCBe100i")
r1(NUCBe.set_user_limit(500, 'imp_window'), "NUCBe500i")

r2(NLinUCBe.set_user_limit(100, 'random'), "NLinUCBe100r")
r2(NLinUCBe.set_user_limit(500, 'random'), "NLinUCBe500r")
r2(NLinUCBe.set_user_limit(100, 'click_window'), "NLinUCBe100c")
r2(NLinUCBe.set_user_limit(500, 'click_window'), "NLinUCBe500c")
r2(NLinUCBe.set_user_limit(100, 'imp_window'), "NLinUCBe100i")
r2(NLinUCBe.set_user_limit(500, 'imp_window'), "NLinUCBe500i")

# experiment7: different similarities
r1(NUCBAgent(0.1, 100, 100).set_similarity_type('c'), "NUCBc")
r2(NLinUCBAgent(0.1, 100).set_similarity_type('w'), "NLinUCBw")

## Create plots

In [ ]:
from matplotlib import colormaps
c = colormaps['tab20b'].colors[16:20]

alpha = 0.3

def start_plot1(vis, dataset):
    vis.new_plot(dataset, 200000)
    vis.add_result("Random", {"alpha": alpha, "color": "tab:blue"})
    vis.add_result("UCB",    {"alpha": alpha, "color": "tab:green"})

def start_plot2(vis, dataset):
    vis.new_plot(dataset, 50000)
    vis.add_result("Random",     {"alpha": alpha, "color": "tab:blue"})
    vis.add_result("ONE-LinUCB", {"alpha": alpha, "color": "tab:green"})
    vis.add_result("MUL-LinUCB", {"alpha": alpha, "color": "tab:green", "linestyle": "dashed"})

In [ ]:
# experiment2
vis = ExperimentVisualizer("experiments/raw", "experiments/plots2")

for d in ["lastfmSW", "lastfmRP", "deliciousSW", "deliciousRP"]:
    start_plot1(vis, d)
    vis.add_result("NUCB",  {"label": "NUCB w/ global info", "color": "tab:purple"})
    vis.add_result("NUCBl", {"label": "NUCB w/o global info", "color": "tab:purple", "linestyle": "dashed"})
    vis.create_plot("rewards")

for d in ["lastfmxSW", "lastfmxRP", "deliciousxSW", "deliciousxRP"]:
    start_plot2(vis, d)
    vis.add_result("NLinUCB",  {"label": "NLinUCB w/ global info", "color": "tab:purple"})
    vis.add_result("NLinUCBl", {"label": "NLinUCB w/o global info", "color": "tab:purple", "linestyle": "dashed"})
    vis.create_plot("rewards")

In [ ]:
# experiment3
vis = ExperimentVisualizer("experiments/raw", "experiments/plots3")

for d in ["lastfmSW", "lastfmRP", "deliciousSW", "deliciousRP"]:
    start_plot1(vis, d)
    vis.add_result("NUCB",  {"label": "NUCB w/ normalization", "color": "tab:purple"})
    vis.add_result("NUCBn", {"label": "NUCB w/o normalization", "color": "tab:purple", "linestyle": "dashed"})
    vis.create_plot("rewards")

for d in ["lastfmSW", "lastfmRP", "deliciousSW", "deliciousRP"]:
    start_plot1(vis, d)
    vis.add_result("NUCBn100",   {"label": "NUCB $z_t=1$ $\\beta$=100",   "color": c[3]})
    vis.add_result("NUCBn500",   {"label": "NUCB $z_t=1$ $\\beta$=500",   "color": c[2]})
    vis.add_result("NUCBn1000",  {"label": "NUCB $z_t=1$ $\\beta$=1000",  "color": c[1]})
    vis.add_result("NUCBn10000", {"label": "NUCB $z_t=1$ $\\beta$=10000", "color": c[0]})
    vis.create_plot("rewards", "uw")

for d in ["lastfmxSW", "lastfmxRP", "deliciousxSW", "deliciousxRP"]:
    start_plot2(vis, d)
    vis.add_result("NLinUCB",  {"label": "NLinUCB w/ normalization", "color": "tab:purple"})
    vis.add_result("NLinUCBn", {"label": "NLinUCB w/o normalization", "color": "tab:purple", "linestyle": "dashed"})
    vis.create_plot("rewards")

In [ ]:
# experiment4
vis = ExperimentVisualizer("experiments/raw", "experiments/plots4")

for d in ["lastfmSW", "lastfmRP", "deliciousSW", "deliciousRP"]:
    start_plot1(vis, d)
    vis.add_result("NUCB",     {"label": "NUCB", "color": "tab:purple"})
    vis.add_result("NUCBu100", {"label": "NUCB only user", "color": "tab:purple", "linestyle": "dashed"})
    vis.add_result("NUCBi100", {"label": "NUCB only item", "color": "tab:purple", "linestyle": "dotted"})
    vis.create_plot("rewards", "all")

    start_plot1(vis, d)
    vis.add_result("NUCBu10",   {"label": "NUCB $\\beta$=10",   "color": c[3]})
    vis.add_result("NUCBu100",  {"label": "NUCB $\\beta$=100",  "color": c[2]})
    vis.add_result("NUCBu500",  {"label": "NUCB $\\beta$=500",  "color": c[1]})
    vis.add_result("NUCBu1000", {"label": "NUCB $\\beta$=1000", "color": c[0]})
    vis.create_plot("rewards", "users")

    start_plot1(vis, d)
    vis.add_result("NUCBi10",   {"label": "NUCB $\\beta_a$=10",   "color": c[3]})
    vis.add_result("NUCBi100",  {"label": "NUCB $\\beta_a$=100",  "color": c[2]})
    vis.add_result("NUCBi500",  {"label": "NUCB $\\beta_a$=500",  "color": c[1]})
    vis.add_result("NUCBi1000", {"label": "NUCB $\\beta_a$=1000", "color": c[0]})
    vis.create_plot("rewards", "items")

for d in ["lastfmxSW", "lastfmxRP", "deliciousxSW", "deliciousxRP"]:
    start_plot2(vis, d)
    vis.add_result("NLinUCBu50",   {"label": "NLinUCB $\\beta$=50",   "color": c[3]})
    vis.add_result("NLinUCB",      {"label": "NLinUCB $\\beta$=100",  "color": c[2]})
    vis.add_result("NLinUCBu500",  {"label": "NLinUCB $\\beta$=500",  "color": c[1]})
    vis.add_result("NLinUCBu1000", {"label": "NLinUCB $\\beta$=1000", "color": c[0]})
    vis.create_plot("rewards", "users")

In [ ]:
# experiment 5
vis = ExperimentVisualizer("experiments/raw", "experiments/plots5")

for d in ["lastfmSW", "lastfmRP", "deliciousSW", "deliciousRP"]:
    start_plot1(vis, d)
    vis.add_result("NUCBu100", {"label": "NUCB approximate", "color": "tab:purple", "linestyle": "dotted"})
    vis.add_result("NUCBe",    {"label": "NUCB exact", "color": "tab:purple", "linestyle": "dashed"})
    vis.create_plot("rewards")

for d in ["lastfmxSW", "lastfmxRP", "deliciousxSW", "deliciousxRP"]:
    start_plot2(vis, d)
    vis.add_result("NLinUCB",  {"label": "NLinUCB approximate", "color": "tab:purple", "linestyle": "dotted"})
    vis.add_result("NLinUCBe", {"label": "NLinUCB exact", "color": "tab:purple", "linestyle": "dashed"})
    vis.create_plot("rewards")

In [ ]:
# experiment6
vis = ExperimentVisualizer("experiments/raw", "experiments/plots6")

for d in ["lastfmSW", "lastfmRP", "deliciousSW", "deliciousRP"]:
    start_plot1(vis, d)
    vis.add_result("NUCBe",     {"label": "NUCB", "color": "tab:purple"})
    vis.add_result("NUCBe100r", {"label": "NUCB 100 random", "color": "tab:pink", "linestyle": "dashed"})
    vis.add_result("NUCBe500r", {"label": "NUCB 500 random", "color": "tab:pink", "linestyle": "solid"})
    vis.add_result("NUCBe100c", {"label": "NUCB 100 click queue", "color": "tab:olive", "linestyle": "dashed"})
    vis.add_result("NUCBe500c", {"label": "NUCB 500 click queue", "color": "tab:olive", "linestyle": "solid"})
    vis.add_result("NUCBe100i", {"label": "NUCB 100 imp queue", "color": "tab:cyan", "linestyle": "dashed"})
    vis.add_result("NUCBe500i", {"label": "NUCB 500 imp queue", "color": "tab:cyan", "linestyle": "solid"})
    vis.create_plot("rewards")

for d in ["lastfmxSW", "lastfmxRP", "deliciousxSW", "deliciousxRP"]:
    start_plot2(vis, d)
    vis.add_result("NLinUCBe",     {"label": "NLinUCB", "color": "tab:purple"})
    vis.add_result("NLinUCBe100r", {"label": "NLinUCB 100 random", "color": "tab:pink", "linestyle": "dashed"})
    vis.add_result("NLinUCBe500r", {"label": "NLinUCB 500 random", "color": "tab:pink", "linestyle": "solid"})
    vis.add_result("NLinUCBe100c", {"label": "NLinUCB 100 click queue", "color": "tab:olive", "linestyle": "dashed"})
    vis.add_result("NLinUCBe500c", {"label": "NLinUCB 500 click queue", "color": "tab:olive", "linestyle": "solid"})
    vis.add_result("NLinUCBe100i", {"label": "NLinUCB 100 imp queue", "color": "tab:cyan", "linestyle": "dashed"})
    vis.add_result("NLinUCBe500i", {"label": "NLinUCB 500 imp queue", "color": "tab:cyan", "linestyle": "solid"})
    vis.create_plot("rewards")

In [ ]:
# experiment7
vis = ExperimentVisualizer("experiments/raw", "experiments/plots7")

for d in ["lastfmSW", "lastfmRP", "deliciousSW", "deliciousRP"]:
    start_plot1(vis, d)
    vis.add_result("NUCB",  {"label": "NUCB sim(R)", "color": "tab:purple"})
    vis.add_result("NUCBc", {"label": "NUCB sim(C)", "color": "tab:purple", "linestyle": "dashed"})
    vis.create_plot("rewards")

for d in ["lastfmxSW", "lastfmxRP", "deliciousxSW", "deliciousxRP"]:
    start_plot2(vis, d)
    vis.add_result("NLinUCB",  {"label": "NLinUCB sim(b)", "color": "tab:purple"})
    vis.add_result("NLinUCBw", {"label": "NLinUCB sim(w)", "color": "tab:purple", "linestyle": "dashed"})
    vis.create_plot("rewards")